** Start a collection of tricky cases that the solution should be able to deal with **

** Improve the methods to deal with some of these cases, including:**

Deal with the case where the reporting unit is implied, and the actual subject is a number, i.e.:
'2,536 relocated'

Deal with two quantities in the same sentence relating to different reporting terms and units, i.e.:
'More than 23,000 people have been affected and 2,536 relocated.'

Include households as a reporting unit, i.e.:
'A total of 1,964 households were displaced.'

Include 'affected' as a possible reporting term when certain other keywords are present, for example Storm, Hurricane etc:
'As many as 2,214 households have been affected by the rainstorms'

Extend the subject / object search to include all tokens with dependency pobj, i.e. to cover sentences like:
'A total of 1,964 households were displaced.'
where the main subject is identified as being 'A total'

Deal with the case where a reporting term is part of a conjunction, i.e.:
'16,000 families were affected and evacuated.'

Clean-up articles to deal with missing spacing, i.e.:
'PeriukanIMPACTS42'
'damaged24'

Add inundated as a reporting term.

**There are still plenty of unusual cases to review and deal with **


**JPA Notes 21/02/17**

Search for "Test Articles - jlln" for my test cases.

I've worked through a few extra test cases, and identified (and fixed) some issues. 


Sentences where there are multiple 'facts' being described but with the same reporting unit (eg8,700 homes have been destroyed, 66,000 homes damaged and forced 337,000 people to evacuate) were only extracting one number, which was shared across all reports. 

I was able to modify the get_quantities function to deal with this case:

Because the get_quantity function was comparing token texts, different tokens with the same text were incorrectly considered to be equivalent.
Testing equality of tokens (and therefore testing the membership of a token in a collection) doesn't seem to be properly implemented in spacy. I have tried a few methods to test this, with the best method so far being to compare the .i property (index) of the tokens. 

When choosing the noun to match a verb, in these cases there will be multiple options. I have extended the branch_search_new function to choose the token with the smallest number of descendents in such cases. This has allowed the correct quantity to be extracted for each report in the southern china test article.


Another issue I noticed was the extraction of locations (the first location in the sentence was being used for all reports), but this was easily fixed (just needed to pass the verb to the get_locations functions). A more signi


I also noticed a few (ominous) reports where deaths were occuring in the future.

I found one example where no reports are generated at all, but I wasn't able to figure out why.



In [1]:
import pandas as pd
import spacy
import os
import sys
from nltk import Tree
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from internal_displacement.pipeline import SQLArticleInterface

In [2]:
import textacy
import re

In [246]:
nlp = spacy.load('en')

def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_

In [247]:
pipeline = SQLArticleInterface("../sql_db.sqlite") #Connecting to pre-populated database.
labels,features = pipeline.get_training_data()

In [273]:
person_reporting_terms = [
    'displaced', 'evacuated', 'forced','flee', 'homeless', 'relief camp',
    'sheltered', 'relocated', 'stranded','stuck','stranded',"killed","dead","died","drown"
]

structure_reporting_terms = [
    'destroyed','damaged','swept','collapsed','flooded','washed', 'inundated', 'evacuate'
]

person_reporting_units = ["families","person","people","individuals","locals","villagers","residents","occupants","citizens", "households"]

structure_reporting_units = ["home","house","hut","dwelling","building","shop","business","apartment","flat","residence"]


person_term_lemmas = [t.lemma_ for t in nlp(" ".join(person_reporting_terms))]
structure_term_lemmas = [t.lemma_ for t in nlp(" ".join(structure_reporting_terms))]
person_unit_lemmas = [t.lemma_ for t in nlp(" ".join(person_reporting_units))]
structure_unit_lemmas = [t.lemma_ for t in nlp(" ".join(structure_reporting_units))]

reporting_term_lemmas = person_term_lemmas + structure_term_lemmas
reporting_unit_lemmas = person_unit_lemmas + structure_unit_lemmas

relevant_article_terms = ['Rainstorm', 'hurricane', 'tornado', 'rain', 'storm', 'earthquake']
relevant_article_lemmas = [t.lemma_ for t in nlp(" ".join(relevant_article_terms))]

In [249]:
class Report:
    def __init__(self,locations,date_times,event_term,subject_term,quantity,story):
        self.locations = locations
        if date_times:
            self.date_times = [date for date in date_times]
        else:
            self.date_times = []
        self.event_term = event_term
        self.subject_term = subject_term
        self.quantity = quantity
        self.story = story
    
    def display(self):
        print("Location: {}  DateTime: {}  EventTerm: {}  SubjectTerm:  {}  Quantity: {}"
              .format(self.locations,self.date_times,self.event_term,self.subject_term,self.quantity))
        
    def show_story_tree(self):
        self.display()
        for sentence in nlp(self.story).sents:
            for token in sentence:
                if token.lemma_ == self.event_term:
                    return to_nltk_tree(sentence.root)

In [320]:
def test_token_equality(token_a,token_b):
    if token_a.i == token_b.i:
        return True
    else:
        return False
    
def check_if_collection_contains_token(token,collection):
    for c in collection:
        if test_token_equality(token,c):
            return True
    return False


def get_descendents(sentence,root=None):
    """
    Retrieves all tokens that are descended from the specified root token.
    param: root: the root token
    param: sentence: a span from which to retrieve tokens.
    returns: a list of tokens
    """
    if not root:
        root = sentence.root
    return [t for t in sentence if root.is_ancestor_of(t)]

def get_head_descendents(sentence,root=None):
    """
    Retrieves all tokens that are descended from the head of the specified root token.
    param: root: the root token
    param: sentence: a span from which to retrieve tokens.
    returns: a list of tokens
    """
    if not root:
        root = sentence.root
    else:
        root = root.head
    return [t for t in sentence if root.is_ancestor_of(t)]
    
def check_if_entity_contains_token(tokens,entity):
    """
    Function to test if a given entity contains at least one of a list of tokens.
    param: tokens: A list of tokens
    param: entity: A span
    
    returns: Boolean
    """
    tokens_ = [t.text for t in tokens]
    ret = False
    for token in entity:
        if token.text in tokens_:
            return True
    return False
    

def get_distance_from_root(token,root):
    """
    Gets the parse tree distance between a token and the sentence root.
    :param token: a token
    :param root: the root token of the sentence
    
    returns: an integer distance
    """
    if token == root:
        return 0
    d = 1
    p = token.head
    while p is not root:
        d+=1
        p = p.head
    return d


def get_common_ancestors(tokens):
    ancestors = [set(t.ancestors) for t in tokens]
    if len(ancestors) == 0:
        return []
    common_ancestors = ancestors[0].intersection(*ancestors)
    return common_ancestors    


def get_distance_between_tokens(token_a,token_b):

    if token_b in token_a.subtree:
        distance = get_distance_from_root(token_b,token_a)
    elif token_a in token_b.subtree:
        distance = get_distance_from_root(token_a,token_b)
    else:
        common_ancestors = get_common_ancestors([token_a,token_b])
        distance = 10000
        for ca in common_ancestors:
            distance_a = get_distance_from_root(ca,token_a)
            distance_b = get_distance_from_root(ca,token_b)
            distance_ab = distance_a + distance_b
            if distance_ab < distance:
                distance = distance_ab
    return distance


def get_closest_contiguous_location_block(entity_list,root_node):
    location_entity_tokens = [[token for token in sentence] for sentence in entity_list]
    token_list =  [item for sublist in location_entity_tokens for item in sublist]
    location_tokens_by_distance = sorted([(token,get_distance_between_tokens(token,root_node)) 
                                          for token in token_list],key= lambda x: x[1])
    closest_location = location_tokens_by_distance[0]
    contiguous_block = [closest_token]
    added_tokens = 1
    while added_tokens > 0:
        contiguous_block_ancestors = [[token for token in token_list if token.is_ancestor_of(toke)] for toke in contiguous_block ]
        contiguous_block_subtrees = [token.subtree for token in contiguous_block]
        contiguous_block_neighbours = contiguous_block_ancestors + contiguous_block_subtrees
        contiguous_block_neighbours = [item for sublist in contiguous_block_neighbours for item in sublist]
        added_tokens = 0
        for toke in token_list:
            if not check_if_collection_contains_token(toke,contiguous_block):
                if toke in contiguous_block_neighbours:
                    added_tokens +=1
                    contiguous_block.append(toke)
    return contiguous_block



def get_contiguous_tokens(token_list):
    common_ancestor_tokens = get_common_ancestors(token_list)
    highest_contiguous_block = []
    for toke in token_list:
        if check_if_collection_contains_token(toke.head,common_ancestor_tokens):
            highest_contiguous_block.append(toke)
    added_tokens = 1
    while added_tokens > 0:
        added_tokens = 0
        for toke in token_list:
            if check_if_collection_contains_token(toke.head,highest_contiguous_block):
                if not check_if_collection_contains_token(toke,highest_contiguous_block):
                    highest_contiguous_block.append(toke)
                    added_tokens +=1
    return highest_contiguous_block

def match_entities_in_block(entities,token_block):
    matched = []
    text_block = [t.text for t in token_block] #For some reason comparing identity on tokens does not always work.
    for e in entities:
        et = [t.text for t in e]
        et_in_b = [t for t in et if t in text_block]
        if len(et_in_b) == len(et):
            matched.append(e)
    return matched

def extract_locations(sentence,root=None):
    """
    Examines a sentence and identifies if any of its constituent tokens describe a location.
    If a root token is specified, only location tokens below the level of this token in the tree will be examined. 
    If no root is specified, location tokens will be drawn from the entirety of the span.
    param: sentence       a span
    param: root           a token
    returns: A list of strings, or None
    """

    if not root:
        root = sentence.root
    descendents = get_descendents(sentence,root)
    location_entities = [e for e in nlp(sentence.text).ents if e.label_ == "GPE"]
    if len(location_entities) > 0:
        descendent_location_tokens = []
        for location_ent in location_entities:
            if check_if_entity_contains_token(location_ent,descendents):
                descendent_location_tokens.extend([token for token in location_ent])
        contiguous_token_block = get_contiguous_tokens(descendent_location_tokens)

        block_locations = match_entities_in_block(location_entities,contiguous_token_block)
        if len(block_locations) > 0:
            return [location.text for location in block_locations]
        else:
            return location_entities #If we cannot decide which one is correct, choose them all
                                    #and figure it out at the report merging stage.
    else:
        return []
    
    
    
def extract_dates(sentence,root=None):
    """
    Examines a sentence and identifies if any of its constituent tokens describe a date.
    If a root token is specified, only date tokens below the level of this token in the tree will be examined. 
    If no root is specified, date tokens will be drawn from the entirety of the span.
    Unlike the extract dates function (which returns a list of strings),
    this function returns a list of spacy spans. This is because numerical quantities detected in the 
    branch_search need to be checked to ensure they are not in fact parts of a date.
    
    param: sentence       a span
    param: root           a token
    returns: A list of spacy spans
    """
    if not root:
        root = sentence.root
    descendents = get_head_descendents(sentence,root)
    date_entities = [e for e in nlp(sentence.text).ents if e.label_ == "DATE"]
    if len(date_entities) > 0:
        descendent_date_tokens = []
        for date_ent in date_entities:
            if check_if_entity_contains_token(date_ent,descendents):
                descendent_date_tokens.extend([token for token in date_ent])
        contiguous_token_block = get_contiguous_tokens(descendent_date_tokens)

        block_dates = match_entities_in_block(date_entities,contiguous_token_block)
        return block_dates
    else:
        return None


def basic_number(token):
    if token.text in ("dozens", "hundreds", "thousands"):
        return True
    if token.like_num:
        return True
    else:
        return False

In [322]:
def process_sentence_new(sentence, dates_memory, locations_memory, story):
    """
    Extracts the main verbs from a sentence as a starting point
    for report extraction.
    """
    sentence_reports = []
    # Find the verbs
    main_verbs = textacy.spacy_utils.get_main_verbs_of_sent(sentence)
    for v in main_verbs:
        unit_type, verb_lemma = verb_relevance(v, story)
        if unit_type:
            reports = branch_search_new(v, verb_lemma, unit_type, dates_memory, locations_memory, sentence, story)
            sentence_reports.extend(reports)
    return sentence_reports

def article_relevance(article):
    for token in article:
        if token.lemma_ in relevant_article_lemmas:
            return True

def verb_relevance(verb, article):
    """
    Checks a verb for relevance by:
    1. Comparing to structure term lemmas
    2. Comparing to person term lemmas
    3. Looking for special cases such as 'leave homeless'
    """
    if verb.lemma_ in structure_term_lemmas:
        return structure_unit_lemmas, verb.lemma_
    elif verb.lemma_ in person_term_lemmas:
        return person_unit_lemmas, verb.lemma_
    elif verb.lemma_ == 'leave':
        children = verb.children
        obj_predicate = None
        for child in children:
            if child.dep_ in ('oprd', 'dobj'):
                obj_predicate = child
        if obj_predicate:
            if obj_predicate.lemma_ in structure_term_lemmas:
                return structure_unit_lemmas, 'leave ' + obj_predicate.lemma_
            elif obj_predicate.lemma_ in person_term_lemmas:
                return person_unit_lemmas, 'leave ' + obj_predicate.lemma_
    elif verb.lemma_ == 'affect' and article_relevance(article):
        return structure_unit_lemmas + person_unit_lemmas , verb.lemma_
                
        
    return None, None

def get_quantity_from_phrase(phrase):
    """
    Look for number-like tokens within noun phrase.
    """
    for token in phrase:
        if basic_number(token):
            return token
            
def get_quantity(sentence, unit):
    """
    Split a sentence into noun phrases.
    Search for quantities within each noun phrase.
    If the noun phrase is part of a conjunction, then
    search for quantity within preceding noun phrase
    """
    noun_phrases = list(nlp(sentence.text).noun_chunks)
    # Case one - see if phrase contains the unit
    for i, np in enumerate(noun_phrases):
        if check_if_collection_contains_token(unit,np):
            if unit.dep_ == 'conj':
                return get_quantity_from_phrase(noun_phrases[i-1])
            else:
                return get_quantity_from_phrase(np)
    #Case two - get any numeric child of the unit noun.
    for child in unit.children:
        if basic_number(child):
            return child
    

def simple_subjects_and_objects(verb):
    verb_objects = textacy.spacy_utils.get_objects_of_verb(verb)
    verb_subjects = textacy.spacy_utils.get_subjects_of_verb(verb)
    verb_objects.extend(verb_subjects)
    return verb_objects
            
def get_subjects_and_objects(story, sentence, verb):
    """
    Identify subjects and objects for a verb
    Also check if a reporting unit directly precedes
    a verb and is a direct or prepositional object
    """
    # Get simple or standard subjects and objects
    verb_objects = simple_subjects_and_objects(verb)
    # Special Cases

    #see if unit directly precedes verb
    if verb.i > 0:
        preceding = story[verb.i - 1]
        if preceding.dep_ in ('pobj', 'dobj') and preceding not in verb_objects:
            verb_objects.append(preceding)

    # See if verb is part of a conjunction
    if verb.dep_ == 'conj':
        lefts = list(verb.lefts)
        if len(lefts) > 0:
            for token in lefts:
                if token.dep_ in ('nsubj', 'nsubjpass'):
                    verb_objects.append(token)
        else:            
            ancestors = verb.ancestors
            for anc in ancestors:
                verb_objects.extend(simple_subjects_and_objects(anc))
            
    # Look for 'pobj' in sentence
    if verb.dep_ == 'ROOT':
        for token in sentence:
            if token.dep_ == 'pobj':
                verb_objects.append(token)
    
    return list(set(verb_objects))
            
def branch_search_new(verb, verb_lemma, search_type, dates_memory, locations_memory, sentence, story):
    """
    Extract reports based upon an identified verb (reporting term).
    Extract possible locations or use most recent locations
    Extract possible dates or use most recent dates
    Identify reporting unit by looking in objects and subjects of reporting term (verb)
    Identify quantity by looking in noun phrases.
    """
    possible_locations = extract_locations(sentence,verb)
    possible_dates = extract_dates(sentence)
    if not possible_locations:
        possible_locations = locations_memory
    if not possible_dates:
        possible_dates = dates_memory
    reports = []
    quantity = None
    verb_objects = get_subjects_and_objects(story, sentence, verb)
    #If there are multiple possible nouns and it is unclear which is the correct one
    #choose the one with the fewest descendents. A verb object with many descendents is more likely to 
    #have its own verb as a descendent.
    verb_descendent_counts = [(v,len(list(v.subtree))) for v in verb_objects]
    verb_objects = [x[0] for x in sorted(verb_descendent_counts,key = lambda x: x[1])]
    for o in verb_objects:
        if basic_number(o):
            quantity = o
            if search_type == structure_term_lemmas:
                unit = 'house'
            else:
                unit = 'person'
            report = Report(possible_locations, possible_dates, verb_lemma,
                                    unit, quantity, story.text)
            report.display()
            reports.append(report)
            break
        elif o.lemma_ in search_type:
            # Try and get a number
            quantity = get_quantity(sentence, o)
            report = Report(possible_locations, possible_dates, verb_lemma,
                                    o.lemma_, quantity, story.text)
            reports.append(report)
            report.display()
            break
    return reports

def cleanup(text):
    text = re.sub(r'([a-zA-Z0-9])(IMPACT)', r'\1. \2', text)
    text = re.sub(r'([a-zA-Z0-9])(RESPONSE)', r'\1. \2', text)
    text = re.sub(r'(IMPACT)([a-zA-Z0-9])', r'\1. \2', text)
    text = re.sub(r'(RESPONSE)([a-zA-Z0-9])', r'\1. \2', text)
    text = re.sub(r'([a-zA-Z])(\d)', r'\1. \2', text)
    return text

def process_article_new(story):
    """
    Process a story once sentence at a time
    """
    story = cleanup(story)
    processed_reports = []
    story = nlp(story)
    sentences = list(story.sents) # Split into sentences
    dates_memory = None # Keep a running track of the most recent dates found in articles
    locations_memory = None # Keep a running track of the most recent locations found in articles
    for sentence in sentences: # Process sentence
        reports = []
        reports = process_sentence_new(sentence, dates_memory, locations_memory, story)
        current_locations = extract_locations(sentence)
        if current_locations:
            locations_memory = current_locations
        current_dates = extract_dates(sentence)
        if current_dates:
            dates_memory = current_dates
        processed_reports.extend(reports)
    return list(set(processed_reports))

In [316]:
features = pd.DataFrame(features, columns=['content'])
features = features[~features['content'].isin(['', 'retrieval_failed'])]

In [253]:
def check_language(text):
    try:
        lang = textacy.text_utils.detect_language(text)
        return lang
    except ValueError:
        return 'na'

In [254]:
features['language'] = features['content'].apply(lambda x: check_language(x))

In [255]:
features = features[features['language'] == 'en'].copy()

In [257]:
features['new_reports'] = features['content'].apply(lambda x: process_article_new(x))

Location: ['Afghanistan', 'Pakistan']  DateTime: [this year]  EventTerm: leave dead  SubjectTerm:  person  Quantity: 160
Location: [Afghanistan, Pakistan]  DateTime: [this year]  EventTerm: strand  SubjectTerm:  person  Quantity: dozens
Location: ['Afghanistan', 'Pakistan']  DateTime: [this year]  EventTerm: stick  SubjectTerm:  villager  Quantity: None
Location: ['Sarobi']  DateTime: [this year]  EventTerm: kill  SubjectTerm:  people  Quantity: 61
Location: ['Sarobi']  DateTime: [this year]  EventTerm: wash  SubjectTerm:  home  Quantity: 500
Location: ['Nuristan']  DateTime: [this year]  EventTerm: destroy  SubjectTerm:  home  Quantity: 60
Location: ['Nuristan']  DateTime: [this year]  EventTerm: kill  SubjectTerm:  person  Quantity: one
Location: ['Khost', 'Nangarhar']  DateTime: [this year]  EventTerm: die  SubjectTerm:  people  Quantity: 24
Location: ['Khost', 'Nangarhar']  DateTime: [this year]  EventTerm: destroy  SubjectTerm:  shop  Quantity: None
Location: ['Khost', 'Nangarhar'

In [14]:
features['num_new_reports'] = features['new_reports'].apply(lambda x: len(x))

In [15]:
fail_cases = features[features['num_new_reports'] == 0]

In [16]:
len(fail_cases)

90

In [17]:
fail_cases_text = (c for c in fail_cases['content'].tolist())

**Test Articles - jlln**

In [323]:
article = "Nineteen people are feared dead after violent storms and severe flooding swept the French Riviera, including three people who drowned in a retirement home after a river broke its banks."
reports = process_article_new(article)
for r in reports:
    r.display()
    
#Why does this fail to generate reports?

In [324]:
article = "More than fifty homes and shops were destroyed and thousands of acres of farmland flooded."
reports = process_article_new(article)
for r in reports:
    r.display()

Location: None  DateTime: []  EventTerm: destroy  SubjectTerm:  shop  Quantity: fifty
Location: None  DateTime: []  EventTerm: flood  SubjectTerm:  person  Quantity: thousands
Location: None  DateTime: []  EventTerm: flood  SubjectTerm:  person  Quantity: thousands
Location: None  DateTime: []  EventTerm: destroy  SubjectTerm:  shop  Quantity: fifty


In [325]:
article = "Quoting an official from the Badakhshan provincial government, Xinhua also said that the foods had damaged or destroyed more than 120 houses in the district."
reports = process_article_new(article)
for r in reports:
    r.display()
#Badakhshan is not recognised as a location by spacy. Could retrain the spacy model.

Location: None  DateTime: []  EventTerm: destroy  SubjectTerm:  house  Quantity: 120
Location: None  DateTime: []  EventTerm: destroy  SubjectTerm:  house  Quantity: 120


In [326]:
article = "The June 17 tornado whipped through Essa Township around the supper hour, leaving 100 families homeless while others had to clean up downed trees and debris."
reports = process_article_new(article)
for r in reports:
    r.display()

Location: [Essa Township]  DateTime: [June 17]  EventTerm: leave homeless  SubjectTerm:  family  Quantity: 100
Location: [Essa Township]  DateTime: [June 17]  EventTerm: leave homeless  SubjectTerm:  family  Quantity: 100


In [327]:
article = "Mountainous Afghanistan was the worst hit, with 61 people killed and approximately 500 traditional mud-brick homes washed away in more than a dozen villages in Sarobi, a rural district less than an hour from Kabul, officials said."
reports = process_article_new(article)
for r in reports:
    r.display()

Location: ['Sarobi']  DateTime: []  EventTerm: kill  SubjectTerm:  people  Quantity: 61
Location: ['Sarobi']  DateTime: []  EventTerm: wash  SubjectTerm:  home  Quantity: 500
Location: ['Sarobi']  DateTime: []  EventTerm: wash  SubjectTerm:  home  Quantity: 500
Location: ['Sarobi']  DateTime: []  EventTerm: kill  SubjectTerm:  people  Quantity: 61


In [328]:
#Southern China Test Article - example of a single sentence with multiple facts.
test_article = "Further severe weather, floods and landslides have left 14 people dead and 4 missing in southern China.  Yesterday the Chinese government said that the storms and heavy rainfall from 18 to 22 June 2014 affected nine southern provinces. 8,700 homes have been destroyed, 66,000 homes damaged and forced 337,000 people to evacuate. 42,000 hectares of crops have also been destroyed. Further heavy rainfall is forecast for the next 24 hours.  According to Xinhua:  Seven people have died in central China’s Hunan Province, five in the eastern province of Jiangxi and two in Guangxi Zhuang Autonomous Region, south of the country. Hunan provincial civil affairs bureau confirmed that four people are missing.  Southern China has endured relentless severe weather conditions since 8 May 2014 where 3 people died after flooding in 5 southern provinces.  The flooding continued through May, resulting in a further 19 deaths. Earlier this month 25 people died after floods lasting a week hit much of southern China."
reports = process_article_new(test_article)

Location: ['China']  DateTime: []  EventTerm: leave dead  SubjectTerm:  people  Quantity: 14
Location: ['China']  DateTime: [Yesterday]  EventTerm: destroy  SubjectTerm:  home  Quantity: 8,700
Location: ['China']  DateTime: [Yesterday]  EventTerm: damage  SubjectTerm:  home  Quantity: 66,000
Location: ['China']  DateTime: [Yesterday]  EventTerm: force  SubjectTerm:  people  Quantity: 337,000
Location: [China, Hunan Province, Jiangxi, Guangxi]  DateTime: [the next 24 hours]  EventTerm: die  SubjectTerm:  people  Quantity: None
Location: ['Hunan']  DateTime: [8 May 2014]  EventTerm: die  SubjectTerm:  people  Quantity: 3
Location: ['China']  DateTime: [Earlier this month]  EventTerm: die  SubjectTerm:  people  Quantity: 25


In [264]:
article = "It was early Saturday when a flash flood hit the area and washed away more than 500 houses"
reports = process_article_new(article)
for r in reports:
    r.display()

Location: None  DateTime: [early Saturday]  EventTerm: wash  SubjectTerm:  house  Quantity: 500
Location: None  DateTime: [early Saturday]  EventTerm: wash  SubjectTerm:  house  Quantity: 500


In [270]:
article = "Within hours of the storm, Dowdall had declared a state of emergency and brought in Essa Township emergency departments staff, as well Simcoe County administrators, to assist the 300 people displaced by the storm."
reports = process_article_new(article)
for r in reports:
    r.display()

Location: [Essa Township, Simcoe County]  DateTime: []  EventTerm: displace  SubjectTerm:  people  Quantity: 300
Location: [Essa Township, Simcoe County]  DateTime: []  EventTerm: displace  SubjectTerm:  people  Quantity: 300


Special cases text cleaning:
    words directly followerd by numbers
    KEY words IMPACT and RESPONSE in capitals

In [266]:
article = "'Haikota, 09 August 2013 – Reports indicated that heavy rainfall and the damaging of a major diversion canal in Hashenkit area have caused flooding on the 5th of this month in Haikota sub-zone, thus giving rise to the destruction of 20 residential houses and the washing away of property.  Among the inhabitants, Mr. Mohammed Kerar and Ms. Letemichael Asgedom pointed out that the area has been witnessing fool river floods since the end of July in line with the consistency of heavy rains, and called on the inhabitants in general and herdsmen in particular to practice caution in this connection.  Accordingly, the timing of the flood which occurred at night was a disadvantage for the inhabitants in case of the efforts made to save property.  Mr. Tadese Negash from the Public Technique Service Center in Gash-Barka region disclosed that the available machineries were deployed in the rescuing endeavor.  Mr. Adem Humed, Administrator of Haikota semi-urban center, called for immediate renovation of the destroyed diversion canal, and advised the residents to be vigilant in case of similar incident in the future as heavy showers may continue.      '"
print(cleanup(article))
reports = process_article_new(article)
for report in reports:
    report.display()

'Haikota, 09 August 2013 – Reports indicated that heavy rainfall and the damaging of a major diversion canal in Hashenkit area have caused flooding on the 5th of this month in Haikota sub-zone, thus giving rise to the destruction of 20 residential houses and the washing away of property.  Among the inhabitants, Mr. Mohammed Kerar and Ms. Letemichael Asgedom pointed out that the area has been witnessing fool river floods since the end of July in line with the consistency of heavy rains, and called on the inhabitants in general and herdsmen in particular to practice caution in this connection.  Accordingly, the timing of the flood which occurred at night was a disadvantage for the inhabitants in case of the efforts made to save property.  Mr. Tadese Negash from the Public Technique Service Center in Gash-Barka region disclosed that the available machineries were deployed in the rescuing endeavor.  Mr. Adem Humed, Administrator of Haikota semi-urban center, called for immediate renovation

In [19]:
article = "Costa Rica saw further heavy rainfall and flooding on Monday 06 July and Tuesday 07 July 2015. WMO report that Puerto Limon saw 174.9 mm of rain fall in 24 hours during that period. Costa Rica’s National Meteorological Institute have warned that the heavy rainfall will continue.  Costa Rica’s National Emergency Commission (Comisión Nacional de Prevención de Riesgos y Atención de Emergencias – CNE) issued a red alert for parts of Heredia and Limón provinces.  Authorities say that the rain and flooding has forced a total of 1057 people to evacuate thier homes and move to one of the 13 temporary shelters set up by CNE, many of them in local community halls or churches. Most of those displaced are in the canton of Sarapiqui.  As of 07 July 2015, 94 communities across 15 cantons were facing access problems after roads had been flooded or damaged.  2 Weeks of Flooding  This latest period of heavy rain means that parts of Costa Rica have suffered flooding for over 2 weeks. Floods in the Caribbean and Northern regions on 21 June 2015 left 25 communities isolated and hundreds displaced.  After a short respite, where some of those affected were able to return to thier homes, northern and eastern parts of the country were faced with further torrential rain on 27 June which caused damage to roads, bridges and houses, and affected over 200 communities across 3 provinces of Limón, Heredia and Cartago.  Colombia and Venezuela  The heavy rain is continuing to fall in parts of northern Colombia and Venzuela, both of which have been hit by floods over the last 10 days. Over 50 mm of rain fell in 24 hours between 06 and 07 July in Arauca, Colombia, according to WMO. Guanare in Venezuela saw 53 mm of rain in the same period. A state of emergency was recently declared after heavy rain and floods in the state of Apure, Venezuela."
print(cleanup(article))
reports = process_article_new(article)
for report in reports:
    report.display()

Costa Rica saw further heavy rainfall and flooding on Monday 06 July and Tuesday 07 July 2015. WMO report that Puerto Limon saw 174.9 mm of rain fall in 24 hours during that period. Costa Rica’s National Meteorological Institute have warned that the heavy rainfall will continue.  Costa Rica’s National Emergency Commission (Comisión Nacional de Prevención de Riesgos y Atención de Emergencias – CNE) issued a red alert for parts of Heredia and Limón provinces.  Authorities say that the rain and flooding has forced a total of 1057 people to evacuate thier homes and move to one of the 13 temporary shelters set up by CNE, many of them in local community halls or churches. Most of those displaced are in the canton of Sarapiqui.  As of 07 July 2015, 94 communities across 15 cantons were facing access problems after roads had been flooded or damaged.  2 Weeks of Flooding  This latest period of heavy rain means that parts of Costa Rica have suffered flooding for over 2 weeks. Floods in the Carib

In [267]:
article = "BEIJING, March 31 (Xinhua) -- The Ministry of Civil Affairs has sent 1,000 tents, 2,000 sleeping bags, 2,000 folding beds and 1,000 sets of folding desks and chairs to Jianhe County in southwestern Guizhou Province after it was hit by a 5.5-magnitude earthquake on Monday morning.  No deaths have been reported, though the quake was Guizhou's biggest in terms of magnitude since 1949. More than 23,000 people have been affected and 2,536 relocated.  Provincial authorities have sent teams to help with the rescue work and allocated 1 million yuan (about 162,880 U.S. dollars) and 206 tents for disaster relief."
print(cleanup(article))
reports = process_article_new(article)
for report in reports:
    report.display()

BEIJING, March 31 (Xinhua) -- The Ministry of Civil Affairs has sent 1,000 tents, 2,000 sleeping bags, 2,000 folding beds and 1,000 sets of folding desks and chairs to Jianhe County in southwestern Guizhou Province after it was hit by a 5.5-magnitude earthquake on Monday morning.  No deaths have been reported, though the quake was Guizhou's biggest in terms of magnitude since 1949. More than 23,000 people have been affected and 2,536 relocated.  Provincial authorities have sent teams to help with the rescue work and allocated 1 million yuan (about 162,880 U.S. dollars) and 206 tents for disaster relief.
Location: ['Guizhou']  DateTime: [1949]  EventTerm: affect  SubjectTerm:  people  Quantity: 23,000
Location: ['Guizhou']  DateTime: [1949]  EventTerm: relocate  SubjectTerm:  person  Quantity: 2,536
Location: ['Guizhou']  DateTime: [1949]  EventTerm: relocate  SubjectTerm:  person  Quantity: 2,536
Location: ['Guizhou']  DateTime: [1949]  EventTerm: affect  SubjectTerm:  people  Quantity

In [21]:
article = 'As many as 2,214 households have been affected by the rainstorms in Rio Grande do Sul, the Emergency Management Service reported today (Dec. 28). A total of 1,964 households were displaced. The storms hit forty municipalities.  According to the government of Rio Grande do Sul, the State Coordination for Emergency Management continues to monitor and provide assistance to the impacted municipalities and communities.  Last Saturday (26), President Rousseff flew over the region, which borders Argentina and Uruguay, and announced the provision of $6.6 million to help communities hit by the floods.  This has been the fifth flood this year in the state, and the most severe. The Quaraí river rose a record 15.28 meters. The situation got even worse with the rise of the Uruguay river.  The rainstorm has disrupted rice harvest in the municipality of Quaraí and caused the Quaraí-Artigas international bridge between Brazil and Uruguay to remain closed off for 22 hours.    Translated by Mayra Borges'
print(cleanup(article))
reports = process_article_new(article)
for report in reports:
    report.display()

As many as 2,214 households have been affected by the rainstorms in Rio Grande do Sul, the Emergency Management Service reported today (Dec. 28). A total of 1,964 households were displaced. The storms hit forty municipalities.  According to the government of Rio Grande do Sul, the State Coordination for Emergency Management continues to monitor and provide assistance to the impacted municipalities and communities.  Last Saturday (26), President Rousseff flew over the region, which borders Argentina and Uruguay, and announced the provision of $6.6 million to help communities hit by the floods.  This has been the fifth flood this year in the state, and the most severe. The Quaraí river rose a record 15.28 meters. The situation got even worse with the rise of the Uruguay river.  The rainstorm has disrupted rice harvest in the municipality of Quaraí and caused the Quaraí-Artigas international bridge between Brazil and Uruguay to remain closed off for 22 hours.    Translated by Mayra Borges

In [268]:
# Extend subject / object to search to include all tokens with dep_ pobj

article = 'As many as 2,214 households have been affected by the rainstorms in Rio Grande do Sul, the Emergency Management Service reported today (Dec. 28). A total of 1,964 households were displaced. The storms hit forty municipalities.  According to the government of Rio Grande do Sul, the State Coordination for Emergency Management continues to monitor and provide assistance to the impacted municipalities and communities.  Last Saturday (26), President Rousseff flew over the region, which borders Argentina and Uruguay, and announced the provision of $6.6 million to help communities hit by the floods.  This has been the fifth flood this year in the state, and the most severe. The Quaraí river rose a record 15.28 meters. The situation got even worse with the rise of the Uruguay river.  The rainstorm has disrupted rice harvest in the municipality of Quaraí and caused the Quaraí-Artigas international bridge between Brazil and Uruguay to remain closed off for 22 hours.    Translated by Mayra Borges'
print(cleanup(article))
reports = process_article_new(article)
for report in reports:
    report.display()

As many as 2,214 households have been affected by the rainstorms in Rio Grande do Sul, the Emergency Management Service reported today (Dec. 28). A total of 1,964 households were displaced. The storms hit forty municipalities.  According to the government of Rio Grande do Sul, the State Coordination for Emergency Management continues to monitor and provide assistance to the impacted municipalities and communities.  Last Saturday (26), President Rousseff flew over the region, which borders Argentina and Uruguay, and announced the provision of $6.6 million to help communities hit by the floods.  This has been the fifth flood this year in the state, and the most severe. The Quaraí river rose a record 15.28 meters. The situation got even worse with the rise of the Uruguay river.  The rainstorm has disrupted rice harvest in the municipality of Quaraí and caused the Quaraí-Artigas international bridge between Brazil and Uruguay to remain closed off for 22 hours.    Translated by Mayra Borges

In [23]:
### Cleanup article, deal with cases where reporting term is part of conjunction, add inundated

article = "Verified  Kampong Cham, Kratie, Stung Treng and Kandal  Description  Due to high intensity of rainfall, Mekong River has swell and caused flooding to the surrounding areas. More flooding is expected if the rain continues. The provinces affected so far includes: Kampong Cham, Kratie, Stung Treng and Kandal12 out of Cambodia's 25 cities and provinces are suffering from floods caused by monsoon rains and Mekong River floodingIMPACT45 dead16,000 families were affected and evacuated3,080 houses inundated44,069 hectares of rice field were inundated5,617 hectares of secondary crops were inundatedRESPONSEThe local authorities provided response to the affected communities. More impact assessment is still conducted by provincial and national authorities.The government also prepared 200 units of heavy equipment in Phnom Penh and the provinces of Takeo, Svay Rieng, Oddar Meanchey and Battambang to divert water or mitigate overflows from inundated homes and farmland"
print(cleanup(article))
reports = process_article_new(article)
for report in reports:
    report.display()

Verified  Kampong Cham, Kratie, Stung Treng and Kandal  Description  Due to high intensity of rainfall, Mekong River has swell and caused flooding to the surrounding areas. More flooding is expected if the rain continues. The provinces affected so far includes: Kampong Cham, Kratie, Stung Treng and Kandal. 12 out of Cambodia's 25 cities and provinces are suffering from floods caused by monsoon rains and Mekong River flooding. IMPACT. 45 dead. 16,000 families were affected and evacuated. 3,080 houses inundated. 44,069 hectares of rice field were inundated. 5,617 hectares of secondary crops were inundated. RESPONSE. The local authorities provided response to the affected communities. More impact assessment is still conducted by provincial and national authorities.The government also prepared 200 units of heavy equipment in Phnom Penh and the provinces of Takeo, Svay Rieng, Oddar Meanchey and Battambang to divert water or mitigate overflows from inundated homes and farmland
Location: ['Ca

In [24]:
article = 'Vehicles pass through a submerged bridge in Larkipora area in Anantnag in the Kashmir Valley (Press Trust of India photo)  Srinagar: At least 10 people were buried by mudslides and hundreds of people left their homes in Kashmir after heavy rain over the weekend raised fears of floods in the valley.'
print(cleanup(article))
reports = process_article_new(article)
for report in reports:
    report.display()

Vehicles pass through a submerged bridge in Larkipora area in Anantnag in the Kashmir Valley (Press Trust of India photo)  Srinagar: At least 10 people were buried by mudslides and hundreds of people left their homes in Kashmir after heavy rain over the weekend raised fears of floods in the valley.


In [25]:
doc = nlp(article)
verbs = textacy.spacy_utils.get_main_verbs_of_sent(doc)
for v in verbs:
    print(v, v.lemma_, textacy.spacy_utils.get_objects_of_verb(v), list(v.children))

pass pass [] [Vehicles, through, )]
submerged submerge [] []
buried bury [] [Srinagar, :, people, were, by, left, .]
left leave [homes] [homes, in, raised]
raised raise [fears] [after, rain, fears, in]


In [26]:
article = 'Verified  Blang Pidie and Labuhan Haji  Description  Due to continuous heavy rainfall flooding has occurred in 8 districts in Aceh Province (i.e. Aceh Besar, Aceh Jaya, Nagan Raya, West Aceh, Banda Aceh, South Aceh, Abdya and Aceh Singkil)Based on data on 3rd November, the following subdistricts in Aceh Barat is affected:Johan Pahlawan, Kawe, Meureubo, Samatiga, Woyla, Woyla Barat, Woyla Timur and BubonIMPACT- 47,524 families or 174,711 affected.- 1,206 families or 4,438 people were displaced.additional 11,966 families or 46,142 people displaced (based on latest 3 Nov 2014 report)- 13,505 houses inundated.- Landslide in Paro Mountain area has disrupted the access from Banda Aceh - Meulaboh (national road).RESPONSE- Local disaster management authorities (BPBA Aceh) responded to the situation and continue the evacuation- The Governor of Aceh declared Provincial Calamity.- National Disaster Management Agency (BNPB) provided 98,000 USD in amount of logistic supplies for the victims.- Indonesian Red Cross Society (PMI) mobilized personnel to conduct assessment and evacuation under the coordination of BPBA(Contributor: Ayu Setiadewi, BNPB)'
print(cleanup(article))
reports = process_article_new(article)
for report in reports:
    report.display()

Verified  Blang Pidie and Labuhan Haji  Description  Due to continuous heavy rainfall flooding has occurred in 8 districts in Aceh Province (i.e. Aceh Besar, Aceh Jaya, Nagan Raya, West Aceh, Banda Aceh, South Aceh, Abdya and Aceh Singkil)Based on data on 3rd November, the following subdistricts in Aceh Barat is affected:Johan Pahlawan, Kawe, Meureubo, Samatiga, Woyla, Woyla Barat, Woyla Timur and Bubon. IMPACT- 47,524 families or 174,711 affected.- 1,206 families or 4,438 people were displaced.additional 11,966 families or 46,142 people displaced (based on latest 3 Nov 2014 report)- 13,505 houses inundated.- Landslide in Paro Mountain area has disrupted the access from Banda Aceh - Meulaboh (national road).RESPONSE- Local disaster management authorities (BPBA Aceh) responded to the situation and continue the evacuation- The Governor of Aceh declared Provincial Calamity.- National Disaster Management Agency (BNPB) provided 98,000 USD in amount of logistic supplies for the victims.- Ind

In [27]:
'Verified  Ciamis, Pangandaran and Cianjur.  Description  High rainfall intensity has caused flooding in several districts of West Java Province. From the period 22 – 24 December 2014, BNPB reported flood happened in Ciamis, Pangandaran and Cianjur.IMPACT:- In Ciamis, 375 families / 1,116 people were affected and 374 houses and 700 ha of crops were inundated.- In Pangandaras, 69 families were affected and 41 ha of crops were submerged.- In Cianjur, 79 families / 303 people were affected.RESPONSE:- BPBD of each district has evacuated the victims and provided relief items.'

'Verified  Ciamis, Pangandaran and Cianjur.  Description  High rainfall intensity has caused flooding in several districts of West Java Province. From the period 22 – 24 December 2014, BNPB reported flood happened in Ciamis, Pangandaran and Cianjur.IMPACT:- In Ciamis, 375 families / 1,116 people were affected and 374 houses and 700 ha of crops were inundated.- In Pangandaras, 69 families were affected and 41 ha of crops were submerged.- In Cianjur, 79 families / 303 people were affected.RESPONSE:- BPBD of each district has evacuated the victims and provided relief items.'

In [28]:
'Verified  karo, North Sumatera  Description  Mt. Sinabung errupted again and emits pyroclastic clouds causing several casualtiesThe affected area includes all villages in the radius of 3 km from the volcanoThe casualties includes:16 death1 injured31,739 people evacuatedBased on research and development agency of Indonesian Ministry of Agriculture the impact of this incident to agriculture:50,921 ha amounted to 1.3 - 1.5 trillion IDR or 114 - 130 millions USDSearch and Rescue team will continue to look for more victims'

'Verified  karo, North Sumatera  Description  Mt. Sinabung errupted again and emits pyroclastic clouds causing several casualtiesThe affected area includes all villages in the radius of 3 km from the volcanoThe casualties includes:16 death1 injured31,739 people evacuatedBased on research and development agency of Indonesian Ministry of Agriculture the impact of this incident to agriculture:50,921 ha amounted to 1.3 - 1.5 trillion IDR or 114 - 130 millions USDSearch and Rescue team will continue to look for more victims'

In [29]:
'Verified  East Java  Description  Five municipalities in Sidoarjo, East Java Province have been flooded. It was triggered by the heavy rain.IMPACT- 1,000 houses were submergedRESPONSE- BPBD set up post command and provided assistance.- 2,450 ready-to-eat meal and 1,250 food packages have been distributed to the victims.'

'Verified  East Java  Description  Five municipalities in Sidoarjo, East Java Province have been flooded. It was triggered by the heavy rain.IMPACT- 1,000 houses were submergedRESPONSE- BPBD set up post command and provided assistance.- 2,450 ready-to-eat meal and 1,250 food packages have been distributed to the victims.'

In [30]:
'Verified  Banjar, Hulu Sungai  Description  The overflow of Barito River has created flooding in many villages in Banjar and Hulu Sungai District, Province of South Kalimantan.IMPACT:- 13,941 people were affected and 3949 houses were submerged in Banjar.- In Hulu Sungai,about 3,000 houses and 525 ha of agriculture area were inundated.RESPONSE:- Relief items and logistic supplies have been distributed by local government.'

'Verified  Banjar, Hulu Sungai  Description  The overflow of Barito River has created flooding in many villages in Banjar and Hulu Sungai District, Province of South Kalimantan.IMPACT:- 13,941 people were affected and 3949 houses were submerged in Banjar.- In Hulu Sungai,about 3,000 houses and 525 ha of agriculture area were inundated.RESPONSE:- Relief items and logistic supplies have been distributed by local government.'

In [31]:
'Verified  Pamekasan, Pasuruan  Description  Many villages in Pamekasan and Pasuruan district, East Java Province were flooded due to the torrential rain. Hundreds of families were reportedly affected by the flooding.IMPACTPamekasan district- 906 houses submergedPasuruan district- 1,777 families were affected- 2 schools and 51 ha of agriculture were inundated.RESPONSE- The local authority set up command post and evacuation center for the victims as well as distributed relief supplies.'

'Verified  Pamekasan, Pasuruan  Description  Many villages in Pamekasan and Pasuruan district, East Java Province were flooded due to the torrential rain. Hundreds of families were reportedly affected by the flooding.IMPACTPamekasan district- 906 houses submergedPasuruan district- 1,777 families were affected- 2 schools and 51 ha of agriculture were inundated.RESPONSE- The local authority set up command post and evacuation center for the victims as well as distributed relief supplies.'

In [32]:
'Verified  Musi Rawas Utara, Ogan Hilir  Description  Continuous rain has generated flood which affected many municipalities in Ogan Hilir and Musi Rawas Utara district, Province of South Sumatra.IMPACT- 350 houses submergedRESPONSE- BPBD provided medical services and distributed logistic supplies.'

'Verified  Musi Rawas Utara, Ogan Hilir  Description  Continuous rain has generated flood which affected many municipalities in Ogan Hilir and Musi Rawas Utara district, Province of South Sumatra.IMPACT- 350 houses submergedRESPONSE- BPBD provided medical services and distributed logistic supplies.'

In [33]:
'Verified  Banten  Description  Due to heavy rainfall the city of tangerang and Serang district in Banten experience flooding.This incident cause the following casualties- 2000 evacuated- estimated 3540 people affectedLocal disaster management agency and related stakeholder provided the emergency relief'

'Verified  Banten  Description  Due to heavy rainfall the city of tangerang and Serang district in Banten experience flooding.This incident cause the following casualties- 2000 evacuated- estimated 3540 people affectedLocal disaster management agency and related stakeholder provided the emergency relief'

In [34]:
'Verified  Garut  Description  Torrential rain has caused Cibarengkok River in Garut to overflow. Consequently, water inundated 10 villages in Banjarwangi and Cikajang sub-district, Garut, Province of West Java.IMPACT- over 2,167 people were affected- 458 houses were damaged- 280 houses were submergedRESPONSE- Evacuation have been conducted by the local government- Relief items have been distributed to the affected communities'

'Verified  Garut  Description  Torrential rain has caused Cibarengkok River in Garut to overflow. Consequently, water inundated 10 villages in Banjarwangi and Cikajang sub-district, Garut, Province of West Java.IMPACT- over 2,167 people were affected- 458 houses were damaged- 280 houses were submergedRESPONSE- Evacuation have been conducted by the local government- Relief items have been distributed to the affected communities'

In [35]:
'Verified  Rote Ndao, Malaka  Description  Flood submerged hundreds of houses in Rote Ndao and Malaka District, East Nusa Tenggara Province. The incident was triggered by heavy rain that caused Baa and Benanain River to overflow.IMPACT- In Rote Ndao, the flood has caused 1 death and submerged 185 houses.- In Malaka, 870 houses were inundated.RESPONSE- Evacuation and relief supply distribution have been conducted by the local government to respond the situation.'

'Verified  Rote Ndao, Malaka  Description  Flood submerged hundreds of houses in Rote Ndao and Malaka District, East Nusa Tenggara Province. The incident was triggered by heavy rain that caused Baa and Benanain River to overflow.IMPACT- In Rote Ndao, the flood has caused 1 death and submerged 185 houses.- In Malaka, 870 houses were inundated.RESPONSE- Evacuation and relief supply distribution have been conducted by the local government to respond the situation.'

In [36]:
'Unverified  Kutai Kartanegara  Description  Heavy rain has caused flooding and landslide in Kutai Kartanegara, East Kalimantan Province.IMPACT- 165 families were affected- 165 houses were submerged- road damaged due to landslideRESPONSE- BPBD provided assistance for the victims.'

'Unverified  Kutai Kartanegara  Description  Heavy rain has caused flooding and landslide in Kutai Kartanegara, East Kalimantan Province.IMPACT- 165 families were affected- 165 houses were submerged- road damaged due to landslideRESPONSE- BPBD provided assistance for the victims.'

In [37]:
'Verified  Bogor  Description  Torrential rain has caused flood and landslide in North Bogor and Sukaraja sub-district in Bogor, Province of West Java.IMPACT- 2 death due to landslide- 113 houses were submergedRESPONSE- Disaster Management Agency of Bogor with the other agencies responded the situation by providing assistance.'

'Verified  Bogor  Description  Torrential rain has caused flood and landslide in North Bogor and Sukaraja sub-district in Bogor, Province of West Java.IMPACT- 2 death due to landslide- 113 houses were submergedRESPONSE- Disaster Management Agency of Bogor with the other agencies responded the situation by providing assistance.'

In [38]:
'Verified  Garut  Description  The overflow of Cimanuk River due to the heavy rain has submerged four subdistricts in Garut, West Java. The affected subdistrict are Tarogong Kidul, Cisurupan, Cikajang and Cikeduk. Consequently, 161 families have been impacted.The detail impact numbers as follows:- 161 families were affected- 162 houses and 1 mosque were submergedBPBD Garut mobilised rescue team and evacuation equipment, such as rubber boat and drainage pump. Logistic supplies were also distributed to the the affected community.'

'Verified  Garut  Description  The overflow of Cimanuk River due to the heavy rain has submerged four subdistricts in Garut, West Java. The affected subdistrict are Tarogong Kidul, Cisurupan, Cikajang and Cikeduk. Consequently, 161 families have been impacted.The detail impact numbers as follows:- 161 families were affected- 162 houses and 1 mosque were submergedBPBD Garut mobilised rescue team and evacuation equipment, such as rubber boat and drainage pump. Logistic supplies were also distributed to the the affected community.'

In [39]:
'Verified  Bone  Description  Flood affected Bone district in South Sulawesi which caused by the overflow of Awu river.IMPACT- 51 houses were submergedRESPONSE- Evacuation and relief operation have been conducted by the BPBD.'

'Verified  Bone  Description  Flood affected Bone district in South Sulawesi which caused by the overflow of Awu river.IMPACT- 51 houses were submergedRESPONSE- Evacuation and relief operation have been conducted by the BPBD.'

In [40]:
'Verified  Pasaman, Padang City  Description  Flashflood happened in Pasaman District and Padang City, West Sumatra Provinci.IMPACT- 1 death- 70 families in Pasaman and 41 families in Padang (216 people) were evacuated- 30 houses and 1 mosque were damagedRESPONSE- Evacuation centre and command post have been set up by BPBD to assist the victims'

'Verified  Pasaman, Padang City  Description  Flashflood happened in Pasaman District and Padang City, West Sumatra Provinci.IMPACT- 1 death- 70 families in Pasaman and 41 families in Padang (216 people) were evacuated- 30 houses and 1 mosque were damagedRESPONSE- Evacuation centre and command post have been set up by BPBD to assist the victims'

In [41]:
'Verified  Bengkulu  Description  Due to heavy rainfall, flood was occurred in several subdistricts in Seluma, Bengkulu. The affected subdistricts includes:Seluma Selatan and Air PeriukanIMPACTS42 families evacuated40 houses inundated15 boats damaged40 ha rice field inundatedRESPONSElocal government stakeholders provided emergency response and reliefhttp://adinet.ahacentre.org/reports/view/739'

'Verified  Bengkulu  Description  Due to heavy rainfall, flood was occurred in several subdistricts in Seluma, Bengkulu. The affected subdistricts includes:Seluma Selatan and Air PeriukanIMPACTS42 families evacuated40 houses inundated15 boats damaged40 ha rice field inundatedRESPONSElocal government stakeholders provided emergency response and reliefhttp://adinet.ahacentre.org/reports/view/739'

In [42]:
'Verified  Bandung  Description  Torrential rain has triggered landslide incidentin Cipatat, West Bandung district, Province of West Java. The reported impacts are 35 houses and 125 people displaced due to the incident. The local authority distributed relief items to the victim and cleaned out the area.'

'Verified  Bandung  Description  Torrential rain has triggered landslide incidentin Cipatat, West Bandung district, Province of West Java. The reported impacts are 35 houses and 125 people displaced due to the incident. The local authority distributed relief items to the victim and cleaned out the area.'

In [43]:
'Verified  Sukabumi  Description  Due to heavy rainfall two sub-districts in Sukabumi experienced flooding. The affected area includes cicurug and cibadak sub-districtsIMPACT1 injured23 houses inundated3 houses collapsedRESPONSESLocal disaster management agency along with related stakeholders provided the relief items and temporary evacuation'

'Verified  Sukabumi  Description  Due to heavy rainfall two sub-districts in Sukabumi experienced flooding. The affected area includes cicurug and cibadak sub-districtsIMPACT1 injured23 houses inundated3 houses collapsedRESPONSESLocal disaster management agency along with related stakeholders provided the relief items and temporary evacuation'

In [44]:
'Verified  West Java  Description  Landslide triggered by heavy rain occurred in Tasikmalaya district, West Java Province.IMPACT- 26 houses were damaged- 28 families or 96 people were evacuatedRESPONSE- BPBD is still conducting damage assessment'

'Verified  West Java  Description  Landslide triggered by heavy rain occurred in Tasikmalaya district, West Java Province.IMPACT- 26 houses were damaged- 28 families or 96 people were evacuatedRESPONSE- BPBD is still conducting damage assessment'